In [1]:
import os
import yaml
import tifffile
import numpy as np

from src import measures
from src.cleaner import DoubleStepCleaner

In [2]:
main_organ = 'brain'
fpath = "/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/"
cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"
db_path = "/home/ws/er5241/Repos/measuring-repo/artifacts/debug.json"

with open(cfg_path, "r") as stream:
    configs = yaml.safe_load(stream)

In [3]:
fls = os.listdir(fpath)
id = np.random.choice(fls)

id = 'Medaka_1105_20-1'
img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"

print(id)

Medaka_1105_20-1


In [4]:
img = tifffile.imread(img_path)
msk = tifffile.imread(msk_path)

In [5]:
cleaning_config = configs['cleaning']
measuring_config = configs['measures']

In [6]:
cleaner = DoubleStepCleaner(**cleaning_config)

msk_clean, roi = cleaner(msk)
img_clean = img[roi]

In [7]:
measure_funcs = ['volume', 'volume_filled',
                 'surface_area',
                  'color_mean', 'color_mean_dilated', 'color_mean_eroded',
                  'color_median', 'color_median_dilated', 'color_median_eroded',
                  'color_std', 'color_std_dilated', 'color_std_eroded',
                  'color_perc_99', 'color_perc_99_dilated', 'color_perc_99_eroded',
                  'color_perc_1', 'color_perc_1_dilated', 'color_perc_1_eroded',
                  'convex_volume',
                  'radius_minimal_sphere', 'radius_maximal_sphere'
                 ]

In [8]:
lbl = measuring_config[5]
label_mask = (msk_clean == lbl['id'])

for measure in measure_funcs:
  mf = getattr(measures, measure)
  cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
  measurement = mf(label_mask, img_clean, cc)
  print(measure, measurement)

volume [263101]
volume_filled [263104]
surface_area [54135]
color_mean [120.98080204940308]
color_mean_dilated [78.94860603270445]
color_mean_eroded [123.44169692898676]
color_median [121.0]
color_median_dilated [69.0]
color_median_eroded [124.0]
color_std [13.041842135983163]
color_std_dilated [33.60336856146502]
color_std_eroded [9.951016508575512]
color_perc_99 [150.0]
color_perc_99_dilated [197.0]
color_perc_99_eroded [144.0]
color_perc_1 [87.0]
color_perc_1_dilated [35.0]
color_perc_1_eroded [96.0]
convex_volume [396452.33333333326]
radius_minimal_sphere [63.735785052904575]
radius_maximal_sphere [5.7626000891584255]
